# Mistral and Weights & Biases

- Weights & Biases: https://wandb.ai/
- Mistral finetuning docs: https://docs.mistral.ai/capabilities/finetuning/
- Tracing with W&B Weave: https://wandb.me/weave

In [1]:
# !pip install mistralai pandas weave

## Using Mistral and Weave

You will probably integrate MistralAI API calls in your codebase by creating a function like the one below:

In [1]:
import os
import weave
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key=os.environ["MISTRAL_API_KEY"])

@weave.op()  # <---- add this and you are good to go
def call_mistral(model:str, messages:list) -> str:
    "Call the Mistral API"
    chat_response = client.chat(
        model=model,
        messages=messages,
    )
    return chat_response.choices[0].message.content

The only thing you need to do is add the @weave.op() decorator to the function you want to trace.

Let's define a more interesting function that recommends cheese based on the region and model.



In [2]:
@weave.op()
def cheese_recommender(region:str, model:str) -> str:
    "Recommend the best cheese in a given region"
     
    messages = [ChatMessage(
        role="user", 
        content=f"What is the best cheese in {region}?")]

    cheeses = call_mistral(model=model, messages=messages)
    return {"region": region, "cheeses": cheeses}

Let's run this function and see how weave traces it. We call weave.init() to tell weave the project where to store the traces.

In [3]:

weave.init("mistral_webinar")
print(cheese_recommender(region="France", model="open-mistral-7b"))

weave version 0.50.5 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/mistral_webinar/weave
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/06a1c478-b778-4c6a-a7ce-4dfabca93a3f
{'region': 'France', 'cheeses': 'France is renowned for its diverse and high-quality cheeses, and it\'s difficult to pinpoint a single "best" cheese as tastes can vary greatly. However, some of the most famous French cheeses include:\n\n1. Roquefort (Blue Cheese): This is one of the most famous French blue cheeses made from sheep\'s milk. It\'s aged in the Combalou caves near Roquefort-sur-Soulzon in the Massif Central.\n\n2. Camembert de Normandie: A soft, creamy, and pungent cow\'s milk cheese originating from Normandy. It\'s known for its white mold rind.\n\n3. Comté: A hard, nutty, and slightly sweet cheese made from unpasteurized cow\'s milk in the Franche-Comté region of eastern France.\n\n4. 

You can view the traces by clicking the link above 👆
![](cheese_recomender.png)



## Prepare the dataset

Some data from wandbot

In [4]:
import pandas as pd
df = pd.read_json('qa.jsonl', orient='records', lines=True)

In [5]:
df.head()

,question,answer
0,What is the difference between team and organi...,A team is a collaborative workspace for a grou...
1,What is the difference between team and entity...,A team is a collaborative workspace for a grou...
2,What is a team and where can I find more infor...,Use W&B Teams as a central workspace for your ...
3,When should I log to my personal entity agains...,You should log to your personal entity when yo...
4,Who can create a team? Who can add or delete p...,**Admin**: Team admins can add and remove othe...


A neat trick to get better answers is instead of passing a very long initial message, passing a small conversation with some prefilled agent responses.

In [6]:
def create_messages(question: str, cls=ChatMessage):
    messages = [
        cls(
            role="user", 
            content=(
                "You are an expert about Weights & Biases the ML platform. "
                 "You will answer questions about the product, Answer the question directly, without repeating the instructions."
                 )
        ),
        cls(
            role="assistant", 
            content=(
                "Sure, I'd be happy to help with your question about Weights & Biases. "
                 "If you have a specific question about using Weights & Biases, such as how to track experiments, "
                 "visualize data, or manage artifacts, please feel free to ask!")
        ),
        cls(
            role="user", 
            content=f"Here is the question: {question}"
        )
    ]
    return messages

In [7]:
@weave.op()
def wandb_expert(question:str, model:str) -> str:
    "Answer questions about wandb"
     
    messages = create_messages(question=question)

    answer = call_mistral(model=model, messages=messages)
    return {"question": question, "answer": answer}

res = wandb_expert(question=df.loc[0].question, model="open-mistral-7b")
print(df.loc[0].question)
print(res["answer"])

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/18104a76-c858-472d-8902-e1d45ccfa8b4
What is the difference between team and organization?
In Weights & Biases (W&B), a "team" is a collection of users who work together on a specific project or set of projects. A team is typically used when multiple people are collaborating on a single research initiative, such as a research group or a machine learning team within a company.

On the other hand, an "organization" is a higher-level entity that contains multiple teams. An organization is used when there are multiple, independent research initiatives or machine learning teams within a company, or when multiple organizations are collaborating on a joint research project.

So, in summary, a team is a group of users working on a specific project or set of projects, while an organization is a collection of teams.


This is a very random answer, without following the instruction nor knowing about the question itself. Let's try `mistral-large` for comparison.

In [8]:
res = wandb_expert(question=df.loc[0].question, model="mistral-large-latest")
print(df.loc[0].question)
print(res["answer"])

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/fb6c7f27-e154-4c98-8570-6d15352bf504
What is the difference between team and organization?
In Weights & Biases, the main difference between a team and an organization is the level of control and collaboration that they offer.

A team is a group of users who can collaborate on experiments and share resources, such as datasets and models. Teams are typically used by small groups of people who are working together on a single project or a set of related projects. Teams have a single owner who can manage membership, billing, and permissions.

An organization, on the other hand, is a larger entity that can contain multiple teams. Organizations are typically used by companies or other large organizations that need to manage multiple teams and projects. Organizations have a more complex hierarchy and offer more advanced features for managing users and resources, such as single sign-on (SSO) and centralized billing.

In summary, teams are suit

This is pretty descent for both 😍. Let's see if fine-tuning improves this.

In [9]:
def format_messages(row):
    "Format on the expected MistralAI fine-tuning dataset"
    question = row['question']
    answer = row['answer']
    messages = create_messages(question, cls=dict)
    # we need to append the answer for training 👇
    messages = {"messages":messages + [dict(role="assistant", content=answer)]}
    return messages

In [10]:
msgs = format_messages(df.loc[0])
msgs

{'messages': [{'role': 'user',
   'content': 'You are an expert about Weights & Biases the ML platform. You will answer questions about the product, Answer the question directly, without repeating the instructions.'},
  {'role': 'assistant',
   'content': "Sure, I'd be happy to help with your question about Weights & Biases. If you have a specific question about using Weights & Biases, such as how to track experiments, visualize data, or manage artifacts, please feel free to ask!"},
  {'role': 'user',
   'content': 'Here is the question: What is the difference between team and organization?'},
  {'role': 'assistant',
   'content': 'A team is a collaborative workspace for a group of users working on the same projects, while an organization is a higher-level entity that may consist of multiple teams and is often related to billing and account management.'}]}

In [11]:
df = df.apply(format_messages, axis=1)
df.head()

0    {'messages': [{'role': 'user', 'content': 'You...
1    {'messages': [{'role': 'user', 'content': 'You...
2    {'messages': [{'role': 'user', 'content': 'You...
3    {'messages': [{'role': 'user', 'content': 'You...
4    {'messages': [{'role': 'user', 'content': 'You...
dtype: object

In [12]:
df_train=df.sample(frac=0.9,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("train.jsonl", orient="records", lines=True)
df_eval.to_json("eval.jsonl", orient="records", lines=True)

## Upload dataset

In [13]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("train.jsonl", "rb") as f:
    ds_train = client.files.create(file=("train.jsonl", f))
with open("eval.jsonl", "rb") as f:
    ds_eval = client.files.create(file=("eval.jsonl", f))


In [14]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [15]:
pprint(ds_train)

{
    "id": "d40cc185-6f0d-4754-bc05-5db7f6e3723a",
    "object": "file",
    "bytes": 147176,
    "created_at": 1719343148,
    "filename": "train.jsonl",
    "purpose": "fine-tune"
}


In [16]:
pprint(ds_eval)

{
    "id": "2a5b6582-3e90-4af6-800e-e9cf2d904bda",
    "object": "file",
    "bytes": 15339,
    "created_at": 1719343148,
    "filename": "eval.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

In [20]:
from mistralai.models.jobs import TrainingParameters, WandbIntegrationIn

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ds_train.id],
    validation_files=[ds_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=25,
        learning_rate=0.0001,
        ),
    integrations=[
        WandbIntegrationIn(
            project="mistral_webinar",
            run_name="finetune_wandb",
            api_key=os.environ.get("WANDB_API_KEY"),
        ).dict()
    ],
)

In [21]:
pprint(created_jobs)

{
    "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
    "hyperparameters": {
        "training_steps": 25,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1719343209,
    "modified_at": 1719343209,
    "training_files": [
        "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
    ],
    "validation_files": [
        "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ]
}


In [22]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
    "hyperparameters": {
        "training_steps": 25,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1719343209,
    "modified_at": 1719343210,
    "training_files": [
        "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
    ],
    "validation_files": [
        "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719343210
        },
        {
            "name": "status-updated",
            "data": {
                "status": "QUEUED"
            },
            "

In [23]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
            "hyperparameters": {
                "training_steps": 25,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:open-mistral-7b:0362203c:20240625:a063d186",
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1719343209,
            "modified_at": 1719343355,
            "training_files": [
                "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
            ],
            "validation_files": [
                "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
            ],
            "object": "job",
            "integrations": [
                {
                    "type": "wandb",
                    "project": "mistral_webinar",
                    "name": null,
                    "run_name": "finetune_wandb"
                }
            ]
        },
        {
            "id": "5e4e

In [24]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
    "hyperparameters": {
        "training_steps": 25,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:open-mistral-7b:0362203c:20240625:a063d186",
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1719343209,
    "modified_at": 1719343355,
    "training_files": [
        "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
    ],
    "validation_files": [
        "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1719343355
        },
        {
            "name": "status-updated",
            "data": {
                "sta

## Use a fine-tuned model

In [25]:
df_eval.iloc[1]

{'messages': [{'role': 'user',
   'content': 'You are an expert about Weights & Biases the ML platform. You will answer questions about the product, Answer the question directly, without repeating the instructions.'},
  {'role': 'assistant',
   'content': "Sure, I'd be happy to help with your question about Weights & Biases. If you have a specific question about using Weights & Biases, such as how to track experiments, visualize data, or manage artifacts, please feel free to ask!"},
  {'role': 'user',
   'content': 'Here is the question: What is the difference between `.log()` and `.summary`?'},
  {'role': 'assistant',
   'content': 'The summary is the value that shows in the table while the log will save all the values for plotting later.\n\nFor example, you might want to call `wandb.log` every time the accuracy changes. Usually, you can just use .log. `wandb.log()` will also update the summary value by default unless you have set the summary manually for that metric\n\nThe scatterplo

In [26]:
wandb_expert(question="What is the difference between `.log()` and `.summary`?", 
             model=retrieved_jobs.fine_tuned_model)

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/eaccb136-f7d7-4191-a97f-2aaf8ebcf90a


{'question': 'What is the difference between `.log()` and `.summary`?',
 'answer': 'Use `.log()` to record scalar values that are not gradients, like loss or accuracy. Use `.summary()` to record scalars that are gradients (also called "Vanilla scalars" in the W&B App). The difference between logging scalars with `.log()` and `.summary()` is that `.summary()` automatically scales the scalars by the magnitude of the largest gradient seen so far for that scalar. This scaling makes it easier to compare different scalars with different units, such as loss (which is typically in the range [0, 1]) and learning rate (which is typically in the range [0, 1e-4]). For example, if you log both loss and learning rate with `.log()`, you\'ll have to manually scale the learning rate to compare it to the loss. If you log both with `.summary()`, W&B will automatically scale the learning rate for you.'}